In [27]:
import pandas as pd


df = pd.read_csv('proj831_data.csv')

df


,uid,mapTitle,initial_corpus,processed_corpus,user_id
0,f7ecc724-285a-4a7c-b87e-b32887a19dc7,ARLEQUIN PIA ANRU,"[{""corpus"": ""ARLEQUIN PIA ANRU"", ""activities"":...","[{""corpus"": ""ARLEQUIN PIA ANRU"", ""activities"":...",2
1,4c7f68e2-6a44-477c-babb-8b1de2745245,tests BeatCorp,"[{""corpus"": ""tests BeatCorp"", ""activities"": [{...","[{""corpus"": ""tests BeatCorp"", ""activities"": [{...",1
2,266d42b3-d973-4a89-b600-67cf06e846e8,Les Tilleuls - 80 Arlequin,"[{""corpus"": ""Les Tilleuls - 80 Arlequin"", ""act...","[{""corpus"": ""Les Tilleuls - 80 Arlequin"", ""act...",2
3,ffbe63e9-65d8-401b-bd5b-df85a3e1e37f,Agrithermic,"[{""corpus"": ""Agrithermic"", ""activities"": [{""ta...","[{""corpus"": ""Agrithermic"", ""activities"": [{""in...",1
4,6848117f-1351-4839-a8b3-e451a58b8d12,timeline,"[{""corpus"": ""timeline"", ""activities"": [{""tasks...","[{""corpus"": ""timeline"", ""activities"": [{""index...",1
5,ab78c764-b044-49c0-aa93-37b05b766c9f,tests scraping Trello,"[{""corpus"": ""tests scraping Trello"", ""activiti...","[{""corpus"": ""tests scraping Trello"", ""activiti...",1


In [28]:
import json
#function that transform str to json
def str_to_json(s):
    return json.loads(s)




In [29]:
import json

json_data=[str_to_json(x) for x in df['processed_corpus']]

#save json data as a json file
with open('proj831_data.json', 'w') as f:
    json.dump(json_data, f)


Arthur

In [30]:
import json
import pandas as pd

def json_to_csv(json_data):
    # Préparer une liste pour stocker les données de chaque tâche
    rows = []

    # Itérer à travers chaque corpus dans le JSON
    for corpus_data in json_data:
        corpus_data=corpus_data[0]
        corpus_name = corpus_data['corpus']
        
        # Itérer à travers chaque activité dans le corpus
        for activity in corpus_data['activities']:
            activity_index = activity['index']
            activity_name = activity.get('activity', 'No activity name')
            activity_objectives = ", ".join(activity.get('objectives', []))
            activity_personal_resources = ", ".join(
                [f"{res['type']}: {res['description']}" for res in activity.get('personnal_resources', [])]
            )
            
            # Extraire les performances si elles existent
            activity_performance = ", ".join([
                f"{perf['value']}{perf['unit']} {perf['criterion']}"
                for perf in activity.get('performances', [])
            ])
            # Extraire la situation si elle existe
            situation = activity.get('situation', 'No situation provided')
                                         
            # Itérer à travers chaque tâche dans l'activité
            for task in activity['tasks']:
                task_data = {
                    'corpus': corpus_name,
                    'activity_index': activity_index,
                    'activity_name': activity_name,
                    'activity_objectives': activity_objectives,
                    'activity_personal_resources': activity_personal_resources,
                    'task_index': task['index'],
                    'date': task['date'],
                    'actors': ", ".join(task['actors']),
                    'scheme': task.get('scheme', 'No scheme'),
                    'actions': ", ".join(task['actions']),
                    'validated': task['validated'],
                    'personal_resources': ", ".join(
                        [f"{res['type']}: {res['description']}" for res in task.get('personnal_resources', [])]
                    ),
                    'environmental_resources': ", ".join(
                        [res['description'] for res in task.get('environmental_resources', []) if res.get('type') is None]
                    ),
                    'performance': activity_performance,
                    'situation': situation
                }
                rows.append(task_data)

    # Créer un DataFrame avec les données des tâches
    df = pd.DataFrame(rows)

    # Écrire le DataFrame dans un fichier CSV
    df.to_csv('output_tasks.csv', index=False)
    
#save json data as a json file
with open('proj831_data.json', 'w') as f:
    json.dump(json_data, f)
    
# Appeler la fonction pour convertir les données en CSV
json_to_csv(json_data)



Actor_personal_ressource et Actor_scheme

In [32]:
import pandas as pd
from sqlalchemy import create_engine

# Configuration de la connexion à la base de données
username = 'root'
host = 'localhost'
port = 3306
dbname = 'proj831'
database_url = f"mysql+pymysql://{username}@{host}:{port}/{dbname}"
engine = create_engine(database_url)

# Chemin vers le fichier CSV
file_path = 'output_tasks.csv'

# Lecture du fichier CSV
df = pd.read_csv(file_path)

def prepare_actor_links(dataframe):
    actor_personal_resource_links = []
    actor_scheme_links = []
    
    for idx, row in dataframe.iterrows():
        actor_uid = idx + 1  # UID de l'acteur basé sur l'index de la ligne
        # Lien acteur-ressource personnelle
        if pd.notna(row['personal_resources']):
            actor_personal_resource_links.append({
                'actor_uid': actor_uid,
                'personal_resource_uid': actor_uid  # Supposons que les UIDs des ressources personnelles correspondent
            })
        
        # Lien acteur-schéma
        if pd.notna(row['scheme']):
            actor_scheme_links.append({
                'actor_uid': actor_uid,
                'scheme_uid': actor_uid  # Supposons que les UIDs des schémas correspondent
            })
    
    return actor_personal_resource_links, actor_scheme_links

# Préparation des liens entre acteurs, ressources personnelles et schémas
actor_personal_resource_links, actor_scheme_links = prepare_actor_links(df)

# Conversion en DataFrame pour insertion
actor_personal_resource_df = pd.DataFrame(actor_personal_resource_links)
actor_scheme_df = pd.DataFrame(actor_scheme_links)

# Insertion dans la base de données
if not actor_personal_resource_df.empty:
    actor_personal_resource_df.to_sql('Actor_Personal_resource', con=engine, if_exists='append', index=False)
else:
    print("No valid actor-personal resource links found to insert.")

if not actor_scheme_df.empty:
    actor_scheme_df.to_sql('Actor_Scheme', con=engine, if_exists='append', index=False)
else:
    print("No valid actor-scheme links found to insert.")


IntegrityError: (pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`proj831`.`actor_personal_resource`, CONSTRAINT `actor_personal_resource_ibfk_1` FOREIGN KEY (`actor_uid`) REFERENCES `actor` (`uid`))')
[SQL: INSERT INTO `Actor_Personal_resource` (actor_uid, personal_resource_uid) VALUES (%(actor_uid)s, %(personal_resource_uid)s)]
[parameters: [{'actor_uid': 1, 'personal_resource_uid': 1}, {'actor_uid': 2, 'personal_resource_uid': 2}, {'actor_uid': 3, 'personal_resource_uid': 3}, {'actor_uid': 4, 'personal_resource_uid': 4}, {'actor_uid': 5, 'personal_resource_uid': 5}, {'actor_uid': 6, 'personal_resource_uid': 6}, {'actor_uid': 7, 'personal_resource_uid': 7}, {'actor_uid': 8, 'personal_resource_uid': 8}  ... displaying 10 of 138 total bound parameter sets ...  {'actor_uid': 175, 'personal_resource_uid': 175}, {'actor_uid': 176, 'personal_resource_uid': 176}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)